1. Instalación y carga de librerías

In [ ]:
!pip install yfinance
!pip install pandas==0.25.1
!pip install matplotlib

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from datetime import date
from datetime import timedelta 
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 1000)

2. Definición de función que calcula Histograma MACD

In [ ]:
today = date.today()
def MACD(ticker, first, last):
  global df
  global database
  df = yf.download(ticker, start=first, end=last)
  database = df[['Close', 'Volume', 'High', 'Low']]
  database.reset_index(level=0, inplace=True)
  database.columns=['ds','y', 'vol', 'high', 'low']
  df = df[['Close', 'Volume']]
  mean = df['Volume'].mean()
  std = df['Volume'].std()
  df['Volume'] = (df['Volume'] - mean)/std * 100
  df.reset_index(level=0, inplace=True)
  df.columns=['ds','y', 'vol']
  actualDate = df['ds'].max()
  period = 12
  sma = df['y'].rolling(period, min_periods=period).mean()
  idx_start = sma.isna().sum() + 1 - period
  idx_end = idx_start + period
  sma = sma[idx_start: idx_end]
  rest = df['y'][idx_end:]
  ema = pd.concat([sma, rest]).ewm(span=period, adjust=False).mean()
  df['exp1'] = ema
  period = 26
  sma = df['y'].rolling(period, min_periods=period).mean()
  idx_start = sma.isna().sum() + 1 - period
  idx_end = idx_start + period
  sma = sma[idx_start: idx_end]
  rest = df['y'][idx_end:]
  ema = pd.concat([sma, rest]).ewm(span=period, adjust=False).mean()
  df['exp2'] = ema
  macd = df['exp1']-df['exp2']
  df['macd'] = macd
  df['exp3'] = macd.ewm(span=9, adjust=False).mean()
  df['histog'] = df['macd'] - df['exp3'] 
  df = df[['ds', 'y', 'vol', 'histog']].dropna()
  return df

In [ ]:
def confirmaciones(ticker, first, last, conf):
  global df
  MACD(ticker, first, last)

  lags = max(3, conf) 

  i = 0
  while i <= (lags):
      colname = 'h_%sb' % (i)
      df[colname] = round(df.histog.shift(i),2)
      i = i + 1

  i = lags
  while i >= 1:
      colname = 'h_%sb' % (i-1)
      colname_anterior = 'h_%sb' % (i)
      colname_fl = 'h_fl_conf_alc_%sb' % (i)
      if i == lags:
        df[colname_fl] = np.where(((df[colname_anterior] < 0) & (df[colname] > df[colname_anterior])), 1, 0)
      else:
        df[colname_fl] = np.where(((df[colname_anterior] > 0) & (df[colname] > df[colname_anterior])), 1, 0)
      i = i - 1
  df['q_conf_alcista']= df.iloc[:, (lags*-1):].sum(axis=1)
  df['fl_conf_alcista']= np.where((df['q_conf_alcista'] == lags), 1, 0)

  i = lags
  while i >= 1:
      colname = 'h_%sb' % (i-1)
      colname_anterior = 'h_%sb' % (i)
      colname_fl = 'h_fl_conf_baj_%sb' % (i)
      if i == lags:
        df[colname_fl] = np.where(((df[colname_anterior] > 0) & (df[colname] < df[colname_anterior])), 1, 0)
      else:
        df[colname_fl] = np.where(((df[colname_anterior] < 0) & (df[colname] < df[colname_anterior])), 1, 0)
      i = i - 1
  df['q_conf_bajista']= df.iloc[:, (lags*-1):].sum(axis=1)
  df['fl_conf_bajista']= np.where((df['q_conf_bajista'] == lags), 1, 0)

  i = 2
  while i >= 1:
      colname = 'h_%sb' % (i-1)
      colname_anterior = 'h_%sb' % (i)
      colname_fl = 'h_fl_rev_alc_%sb' % (i)
      df[colname_fl] = np.where((df['histog'] < 0) & (df[colname] > df[colname_anterior]), 1, 0)
      i = i - 1
  df['q_alcista']= df.iloc[:, (2*-1):].sum(axis=1)
  df['fl_rev_alcista']= np.where((df['q_alcista'] == 2), 1, 0)

  i = 3
  while i >= 1:
      colname = 'h_%sb' % (i-1)
      colname_anterior = 'h_%sb' % (i)
      colname_fl = 'h_fl_rev_baj_%sb' % (i)
      df[colname_fl] = np.where((df['histog'] > 0) & (df[colname] < df[colname_anterior]), 1, 0)
      i = i - 1
  df['q_bajista']= df.iloc[:, (3*-1):].sum(axis=1)
  df['fl_rev_bajista']= np.where((df['q_bajista'] == 3), 1, 0)
  df['y_1f'] = round(df.y.shift(-1),2)
  df = df[['ds', 'y_1f', 'histog', 'fl_conf_alcista', 'fl_conf_bajista', 'fl_rev_alcista', 'fl_rev_bajista']]
  df = df[(df.fl_conf_alcista == 1)|(df.fl_conf_bajista == 1)|(df.fl_rev_alcista == 1)|(df.fl_rev_bajista == 1)]
  df.loc[(df['fl_conf_alcista'] == 1), 'señal'] = 'confirmacion_alcista'
  df.loc[(df['fl_conf_bajista'] == 1), 'señal'] = 'confirmacion_bajista'
  df.loc[(df['fl_rev_alcista'] == 1), 'señal'] = 'reversion_alcista'
  df.loc[(df['fl_rev_bajista'] == 1), 'señal'] = 'reversion_bajista'
  df["señal_anterior"] = df.señal.shift(1)
  df["precio_señal_anterior"] = df.y_1f.shift(1)
  df = df[(df.señal_anterior == 'confirmacion_alcista')]

  df['resultado']= (df.y_1f*0.993)/(df.precio_señal_anterior*1.007) - 1

In [ ]:
full = pd.DataFrame()
for ticker in ("ALUA.BA",
"CRES.BA",
"EDN.BA",
"GGAL.BA",
"MIRG.BA",
"PAMP.BA",
"SUPV.BA",
"TECO2.BA",
"TGSU2.BA",
"YPFD.BA"):
  confirmaciones(ticker, first = "2017-11-01", last = "2020-04-01", conf=1)
  full = full.append(df, ignore_index=True)

full['year'] = pd.DatetimeIndex(full['ds']).year
full['month'] = pd.DatetimeIndex(full['ds']).month

full = (full.groupby(['year', 'month']).agg({'señal_anterior':'count',
                                             'resultado':'mean'
                                        }))
print(full)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
            señal_anterior  resultado
year month                           
2018 1                   4   0.104585
     2                   1  -0.089700
     3                   5  -0.046115
     4                   7  -0.033668
     5                   7   0.010419
     6                   6  -0.077893
     7